# Extract SIC Codes from 10K filings

In [121]:
import re
import pandas as pd
from tqdm import tqdm

def get_sic(year: str, debug: bool = False):
    df = pd.read_csv(
        f"Miguel Mapping/CAR_Filtered/CAR_Filtered/CAR_{year}_filtered.csv",
        dtype={"CIK": str}
    )
    
    df.drop_duplicates(subset='CIK', keep='first', inplace=True)
    
    
    results = []
    base_dir = "C:/Users/kcthi/EcoData_10-X_C/10-X_C_2016-2023/10-X_C_2016-2023/"
    year_dir = os.path.join(base_dir, year)
    if not os.path.isdir(year_dir):
        raise FileNotFoundError(f"Year directory does not exist: {year_dir}")

    for _, row in tqdm(
        df.iterrows(),
        total=df.shape[0],
        desc="Processing filings",
        unit="filing",
        ncols=100,
        mininterval=0.1
    ):
        target_cik = row['CIK'].lstrip('0')
        temp = [target_cik]

        # 1) find all files whose 5th part (index 4) == target_cik
        matched_files = []
        for qtr in os.listdir(year_dir):
            qtr_path = os.path.join(year_dir, qtr)
            if not os.path.isdir(qtr_path):
                continue
            for fname in os.listdir(qtr_path):
                if not fname.lower().endswith('.txt'):
                    continue
                parts = fname.split('_')
                # <<< FIX: use index 4, not 3
                if len(parts) >= 5 and parts[4].lstrip('0') == target_cik:
                    matched_files.append(os.path.join(qtr_path, fname))

        if debug:
            print(f"CIK {target_cik} → matched files: {matched_files}")

        # 2) no file → record None
        if not matched_files:
            temp.append(None)
            results.append(temp)
            continue

        # 3) scan *all* matches and collect all SICs
        sic_codes = set()
        for full_path in matched_files:
            with open(full_path, 'r', encoding='utf-8', errors='ignore') as f:
                chunk = f.read(200_000)
            for line in chunk.splitlines():
                if 'STANDARD INDUSTRIAL CLASSIFICATION:' in line:
                    m = re.search(r'\[(\d{4})\]', line)
                    if m:
                        sic_codes.add(m.group(1))
                    else:
                        m2 = re.search(r'\b(\d{4})\b', line)
                        if m2:
                            sic_codes.add(m2.group(1))
                    break  # done with this file

        # If any found, join as comma-separated string; otherwise None
        if sic_codes:
            temp.append(','.join(sorted(sic_codes)))
        else:
            temp.append(None)

        results.append(temp)

    final_df = pd.DataFrame(results, columns=['CIK', 'sic_code'])
    final_df.to_csv(f"Ctrl_Var/SIC/SIC_Codes_{year}.csv", index=False)
    print(f'SIC Codes for {year} have been extracted and saved.')


In [122]:
get_sic("2016")
get_sic("2017")
get_sic("2018")
get_sic("2019")


Processing filings: 100%|███████████████████████████████████| 2152/2152 [00:45<00:00, 47.42filing/s]


SIC Codes for 2016 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 2118/2118 [01:18<00:00, 27.06filing/s]


SIC Codes for 2017 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 2127/2127 [01:19<00:00, 26.72filing/s]


SIC Codes for 2018 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 2183/2183 [01:18<00:00, 27.67filing/s]

SIC Codes for 2019 have been extracted and saved.


In [123]:
get_sic("2020")
get_sic("2021")
get_sic("2022")
get_sic("2023")

Processing filings: 100%|███████████████████████████████████| 2248/2248 [01:27<00:00, 25.69filing/s]


SIC Codes for 2020 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 2549/2549 [01:38<00:00, 25.95filing/s]


SIC Codes for 2021 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 3035/3035 [01:52<00:00, 26.87filing/s]


SIC Codes for 2022 have been extracted and saved.


Processing filings: 100%|███████████████████████████████████| 2899/2899 [01:57<00:00, 24.70filing/s]

SIC Codes for 2023 have been extracted and saved.


In [128]:
df1 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2016.csv")
df2 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2017.csv")
df3 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2018.csv")
df4 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2019.csv")
df5 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2020.csv")
df6 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2021.csv")
df7 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2022.csv")
df8 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2023.csv")


print(df1.isna().sum())
print("===============================")
print(df2.isna().sum())
print("===============================")
print(df3.isna().sum())
print("===============================")
print(df4.isna().sum())
print("===============================")
print(df5.isna().sum())
print("===============================")
print(df6.isna().sum())
print("===============================")
print(df7.isna().sum())
print("===============================")
print(df8.isna().sum())
35 *8

CIK          0
sic_code    35
dtype: int64
CIK          0
sic_code    35
dtype: int64
CIK          0
sic_code    34
dtype: int64
CIK          0
sic_code    34
dtype: int64
CIK          0
sic_code    33
dtype: int64
CIK          0
sic_code    32
dtype: int64
CIK          0
sic_code    33
dtype: int64
CIK          0
sic_code    29
dtype: int64


280

In [130]:
import pandas as pd

# Load data
df1 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2016.csv")
df2 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2017.csv")
df3 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2018.csv")
df4 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2019.csv")
df5 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2020.csv")
df6 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2021.csv")
df7 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2022.csv")
df8 = pd.read_csv("Ctrl_Var/SIC/SIC_Codes_2023.csv")

# Combine all dataframes into a list
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]

# Extract CIKs where sic_code is NaN
missing_sic_df = pd.concat([df[df['sic_code'].isna()][['CIK']] for df in dfs])

# Drop duplicate CIKs
missing_sic_df = missing_sic_df.drop_duplicates().reset_index(drop=True)

# Show the result
missing_sic_df



CIK
0     17313
1     81955
2    817473
3   1143513
4   1259429
5   1278752
6   1287032
7   1287750
8   1297704
9   1321741
10  1326003
11  1370755
12  1372807
13  1414932
14  1418076
15  1444706
16  1464963
17  1476765
18  1487428
19  1487918
20  1490013
21  1490349
22  1496099
23  1508171
24  1509470
25  1509892
26  1512931
27  1513363
28  1552198
29  1559909
30  1571329
31  1577791
32  1578348
33  1578620
34  1587987
35   863894
36  1675033
37  1544206
38  1633336
39  1786108
40  1653384
41  1843162
42    90168

In [2]:

import pandas as pd

# Files and their corresponding years
files = {
    "Ctrl_Var/SIC/SIC_Codes_2016.csv": 2016,
    "Ctrl_Var/SIC/SIC_Codes_2017.csv": 2017,
    "Ctrl_Var/SIC/SIC_Codes_2018.csv": 2018,
    "Ctrl_Var/SIC/SIC_Codes_2019.csv": 2019,
    "Ctrl_Var/SIC/SIC_Codes_2020.csv": 2020,
    "Ctrl_Var/SIC/SIC_Codes_2021.csv": 2021,
    "Ctrl_Var/SIC/SIC_Codes_2022.csv": 2022,
    "Ctrl_Var/SIC/SIC_Codes_2023.csv": 2023,
}

dfs = []
for path, year in files.items():
    df = pd.read_csv(path)
    # add the year column
    df['Year'] = year
    dfs.append(df)

# Extract CIKs where sic_code is NaN, _including_ the Year
missing_sic_df = pd.concat(
    [df.loc[df['sic_code'].isna(), ['CIK', 'Year']] for df in dfs],
    ignore_index=True
)

# If you only want each (CIK, Year) once:
missing_sic_df = missing_sic_df.drop_duplicates().reset_index(drop=True)

# Show the result
missing_sic_df


CIK  Year
0      17313  2016
1      81955  2016
2     817473  2016
3    1143513  2016
4    1259429  2016
..       ...   ...
260  1578348  2023
261  1633336  2023
262  1653384  2023
263  1675033  2023
264  1786108  2023

[265 rows x 2 columns]

In [7]:
year = '2016'

df = pd.read_csv('Ctrl_Var/sp500_daily_return.csv')

# Step 1: Convert 'Filing Date' to datetime
df['Filing Date'] = pd.to_datetime(df['Filing Date'])

# Step 2: Filter for year 2016
df_2016 = df[df['Filing Date'].dt.year == int(year)]

df_2016


CIK Filing Date  S&P500 Return
0        2098  2016-03-11      -0.001261
1        2178  2016-03-11      -0.001261
2        2186  2016-03-02       0.003499
3        2488  2016-02-18      -0.000026
4        3197  2016-03-22      -0.006386
...       ...         ...            ...
2147  1652044  2016-02-11       0.019518
2148  1652101  2016-02-26      -0.008121
2149  1652106  2016-09-26       0.006444
2150  1652923  2016-03-18       0.000986
2151  1667313  2016-10-26      -0.002987

[2152 rows x 3 columns]

In [9]:
test = pd.read_csv('Miguel Mapping/CAR_Filtered/CAR_Filtered/CAR_2016_filtered.csv')
test.drop_duplicates(subset='CIK')

Filing Date      CIK              Window  Expected Return  \
0       11/3/2016     2098  1_day_before_after          -0.8275   
5       11/3/2016     2178  1_day_before_after           0.7448   
10       2/3/2016     2186  1_day_before_after          -6.7155   
15      18/2/2016     2488  1_day_before_after           2.1923   
20      22/3/2016     3197  1_day_before_after          -2.7468   
...           ...      ...                 ...              ...   
10664   11/2/2016  1652044  1_day_before_after          -0.0481   
10669   26/2/2016  1652101  1_day_before_after           0.0920   
10674   26/9/2016  1652106  1_day_before_after           0.0953   
10679   18/3/2016  1652923  1_day_before_after           6.1330   
10684  26/10/2016  1667313  1_day_before_after           0.4065   

       Actual Return  Abnormal Return   CAR  
0             0.0000            0.8275  NaN  
5             0.0000           -0.7448  NaN  
10            0.0000            6.7155  NaN  
15            0.0000           -2.1923  NaN  
20           -0.0612            2.6856  NaN  
...              ...               ...  ...  
10664         0.0008            0.0489  NaN  
10669         0.0010           -0.0910  NaN  
10674        -0.0023           -0.0976  NaN  
10679         0.0000           -6.1330  NaN  
10684         0.0133           -0.3932  NaN  

[2152 rows x 7 columns]

In [ ]:
import pandas as pd

years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

for year in years:
    current_df = pd.read_csv(f'Ctrl_Var/SIC/SIC_Codes_{year}.csv')
    
    current_df['sic_code'] = current_df['sic_code'].fillna(0)
    
    current_df.to_csv(f'SIC_2/SIC_Codes_{year}.csv', index=False)
    
print('All SIC have been updated.')
    

In [4]:
import pandas as pd
import os

years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

for year in years:
    file_path = f'Ctrl_Var/SIC/SIC_Codes_{year}.csv'
    output_path = f'Ctrl_Var/SIC_2/SIC_Codes_{year}.csv'
    
    try:
        current_df = pd.read_csv(file_path)

        if 'sic_code' not in current_df.columns:
            raise KeyError(f"'sic_code' column not found in {file_path}")

        # Step 1: Fill NaN with '0' and convert to string
        current_df['sic_code'] = current_df['sic_code'].fillna('0').astype(str)

        # Step 2: Extract last code if multiple, strip, convert to float, then int
        def clean_sic(s):
            try:
                last_code = s.strip().split(',')[-1]
                return int(float(last_code))  # handles '3420.0' and '2860,6770'
            except:
                return 0  # fallback if parsing fails

        current_df['sic_code'] = current_df['sic_code'].apply(clean_sic)

        # Final check
        if current_df.isna().any().any():
            raise ValueError(f"NaN values remain in {file_path} after processing.")
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        current_df.to_csv(output_path, index=False)
        print(f"✅ Processed {year} successfully.")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

print('🎯 All files attempted.')


✅ Processed 2016 successfully.
✅ Processed 2017 successfully.
✅ Processed 2018 successfully.
✅ Processed 2019 successfully.
✅ Processed 2020 successfully.
✅ Processed 2021 successfully.
✅ Processed 2022 successfully.
✅ Processed 2023 successfully.
🎯 All files attempted.
